  Using cached regex-2024.11.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
Using cached regex-2024.11.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (796 kB)
ERROR: Could not install packages due to an OSError: [Errno 38] Function not implemented: '/opt/conda/lib/python3.12/site-packages/regex-2024.11.6.dist-info'



In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "microsoft/Phi-3-mini-4k-instruct"
# model_name = "Qwen/Qwen2.5-0.5B"
# model_name = "Qwen/Qwen2.5-0.5B-instruct"

# model_name = "KrithikV/MedMobile"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = False)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code = False)

print(model.config.torch_dtype)
print(model.dtype)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("cuda available:", torch.cuda.is_available())
print("device:", device)

print(model.config.torch_dtype)
print(model.dtype)
print(model.name_or_path)

ModuleNotFoundError: No module named 'transformers'

In [2]:
from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en")["train"]
print(len(dataset))

25371


In [ ]:
chosen_data = dataset.select(range(0, 2))
question = dataset[0]["Question"]
# A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?
inputs = tokenizer(question, return_tensors = "pt", padding=False).to(device)

outputs = model.generate(
                inputs["input_ids"],
                max_new_tokens = 500,
                attention_mask=inputs["attention_mask"],  # Use attention mask
                pad_token_id=tokenizer.eos_token_id,  # Set pad token ID
                do_sample = True)
output_text = tokenizer.batch_decode(outputs, skip_special_tokens = True)[0]

print(output_text)

In [9]:
# Test apply_chat_template // https://huggingface.co/docs/transformers/main/en/chat_templating

def format_choices(choices):
    a = zip(list(choices.keys()), choices.values())
    final_answers = []
    for x,y in a:
        final_answers.append(f'[{x}] : {y}')
    return "\n".join(final_answers)

prompt_template = f'''
{{question}} \n
{{choices}}
'''

question = "A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?"
choices = {
"A": "Ampicillin",
"B": "Ceftriaxone",
"C": "Doxycycline",
"D": "Nitrofurantoin"
}

formated_choices = format_choices(choices)

model_prompt = prompt_template.format(question = question, choices = formated_choices)

model_prompt += "Given four answer candidates, A, B, C, D, choose the best answer choice and provide explanation.\n"
model_prompt += "Provide intermediate steps to solve the problem.\n"
model_prompt += "The answer is:\n"

print(model_prompt)

inputs = tokenizer(model_prompt, return_tensors = "pt", padding=False).to(device)

outputs = model.generate(
                inputs["input_ids"],
                max_new_tokens = 500,
                attention_mask=inputs["attention_mask"],  # Use attention mask
                pad_token_id=tokenizer.eos_token_id,  # Set pad token ID
                do_sample = True)
output_text = tokenizer.batch_decode(outputs, skip_special_tokens = True)[0]

print(output_text)


A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient? 

[A] : Ampicillin
[B] : Ceftriaxone
[C] : Doxycycline
[D] : Nitrofurantoin
Given four answer candidates, A, B, C, D, choose the best answer choice and provide explanation.
Provide intermediate steps to solve the problem.
The answer is:


A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and 

In [ ]:
a = {"a":["a1", "a2"], "b":["b1", "b2"]}

ret = []
for i in range(len(a["a"])):
    ret.append(a["a"][i] + a["b"][i])
print(ret)
# print([x for x in a["a"]])
# print(a["a"] + a["b"])

In [6]:
# prepare data on dataset

from datasets import load_dataset
from transformers import Trainer, TrainingArguments

# ultraMedical = load_dataset("TsinghuaC3I/UltraMedical")

# data_list = ultraMedical["train"]
# print(len(data_list), type(data_list))

data_list = dataset

# print((data_list[0:10]))
first_N_data = data_list.select(range(0, 100)) # TODO

print(type(first_N_data))

def preprocess_function(data):
    
#     prompt_added_text = """
# Given four answer candidates, A, B, C, D, choose the best answer choice and provide explanation.
# Provide intermediate steps to solve the problem.
# The answer is:
# """

#     input_text = [conversation[0]['value'] + prompt_added_text + conversation[1]['value'] for conversation in data['conversations']]

    # For medical-o1-reasoning-SFT dataset only
    input_text = []
    for i in range(len(data["Question"])):
        input_text.append(data["Question"][i] + '\n' + data["Response"][i])

    # print(input_text)
    
    tokenized_inputs = tokenizer(input_text, padding="max_length", truncation=True, max_length=1024)

    tokenized_inputs["labels"] = tokenized_inputs["input_ids"][:]
    # tokenized_inputs["labels"] = [ids[1:] + [tokenizer.pad_token_id] for ids in tokenized_inputs["input_ids"]] # Shifted inputs as labels

    # print(tokenized_inputs)
    
    tokenized_inputs = {k: torch.tensor(v) for k, v in tokenized_inputs.items()}

    # print(tokenized_inputs["input_ids"].shape, tokenized_inputs["attention_mask"].shape, tokenized_inputs["labels"].shape)
    
    return tokenized_inputs
    

tokenized_train = first_N_data.map(preprocess_function, batched=True)


<class 'datasets.arrow_dataset.Dataset'>


In [7]:
# Train model on dataset

import shutil
shutil.rmtree("/home/sheju347/Documents/fine_tuned_model/", ignore_errors=True)
print ("previous model removed.")

training_args = TrainingArguments(
    output_dir = "./Documents/fine_tuned_model_checkpoints",
    save_strategy = "no", # TODO, now don't save checkpoints #"epoch",
    per_device_train_batch_size = 1, # As specified in the paper: batch_size: 32
    num_train_epochs = 3, # As specified in the paper: 3 epochs
    learning_rate = 1e-4, # As specified in the paper: 1e-4
    # save_total_limit = 1, # TODO, now keep only the last checkpoint
    fp16 = True, # TODO
    logging_steps = 1,
    logging_dir = "./Documents/logs",  # Directory for logs
    
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_train
)

trainer.train()

# Save the final model
output_dir = "./Documents/fine_tuned_model"

model.to(torch.bfloat16)  # Convert to bfloat16
model.save_pretrained(output_dir, 
                        # save_function=torch.save,  # Use standard PyTorch save
                        # state_dict=model.state_dict(),  # Only save the model weights
                        # safe_serialization=True,  # More efficient serializationsave_optimizer_state=False
                     )
tokenizer.save_pretrained(output_dir,
                         # legacy_format=False  # Use newer, more efficient format
                     )


previous model removed.


ERROR: ld.so: object '/usr/lib64/libstdc++.so.6' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.


Step,Training Loss
1,8.149600
2,8.172600
3,7.317100
4,8.029900
5,8.094900
6,8.852200
7,6.611700
8,8.020500
9,0.572700
10,0.332700


('./Documents/fine_tuned_model/tokenizer_config.json',
 './Documents/fine_tuned_model/special_tokens_map.json',
 './Documents/fine_tuned_model/tokenizer.json')

In [8]:

model = AutoModelForCausalLM.from_pretrained(output_dir, torch_dtype=torch.bfloat16).to(device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  5.57it/s]
